# Google Analytics Customer Revenue Prediction

In [7]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
import os
import matplotlib.pyplot as plt

Define a function to load the CSV files

In [ ]:
# https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields/notebook
def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(
        csv_path,
        converters={ column: json.loads for column in JSON_COLUMNS },
        dtype={ 'fullVisitorId': 'str', 'visitId': 'str' },
        nrows=nrows
    )
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f'{column}.{subcolumn}' for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f'Loaded {os.path.basename(csv_path)}. Shape: {df.shape}')
    return df

In [ ]:
# %%time
# df_train = load_df()
# df_test = load_df('../input/test.csv')

In [ ]:
# %%time
# df_train.to_csv('train-flattened.csv', index=False)
# df_test.to_csv('test-flattened.csv', index=False)

In [25]:
df_train = pd.read_csv('../input/ga-revenue-prediction/train-flattened.csv')
df_test = pd.read_csv('../input/ga-revenue-prediction/test-flattened.csv');

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Target variable
`totals.transactionRevenue`
1. Convert to float, fill NA with 0
2. Only 11515 records resulted in revenue, the remaining 892138 records had zero revenue.
3. Of these, only 9996 visitors (out of 714167 unique visitors) had non-zero revenue. This translates that only 1.4% of visitors bought something.

In [14]:
df_train['totals.transactionRevenue'] = df_train['totals.transactionRevenue'].astype(float)
df_train['totals.transactionRevenue'].fillna(0, inplace=True)
np.sum(df_train['totals.transactionRevenue'] > 0), np.sum(df_train.groupby('fullVisitorId')['totals.transactionRevenue'].sum() > 0)

(11515, 10114)

## Predictors
1. There are 54 predictos in the train set and 52 in the test set.
2. Remove predictors not present in the test set
3. Remove predictors where values are the same for all observations



In [74]:
columns = set(df_train.columns).intersection((set(df_test.columns))) # columns present in both train and test
print('Remove columns:', end=' ')
for i in df_train.columns[df_train.nunique(dropna=False) == 1]:
    print(i, end=', ')
    columns.remove(i)

Remove columns: socialEngagementType, device.browserSize, device.browserVersion, device.flashVersion, device.language, device.mobileDeviceBranding, device.mobileDeviceInfo, device.mobileDeviceMarketingName, device.mobileDeviceModel, device.mobileInputSelector, device.operatingSystemVersion, device.screenColors, device.screenResolution, geoNetwork.cityId, geoNetwork.latitude, geoNetwork.longitude, geoNetwork.networkLocation, totals.visits, trafficSource.adwordsClickInfo.criteriaParameters, 